In [1]:
!pip install evaluate
!pip install transformers
!pip install datasets
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 25.8 MB/s eta 0:00:00


In [2]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import EarlyStoppingCallback
import evaluate
import optuna

In [3]:
# Load the CASP11 dataset
casp11_data = pd.read_csv('/content/2018-06-06-pdb-intersect-pisces.csv')

# Display first few rows of the dataset
casp11_data.head()

,pdb_id,chain_code,seq,sst8,sst3,len,has_nonstd_aa,Exptl.,resolution,R-factor,FreeRvalue
0,1FV1,F,NPVVHFFKNIVTPRTPPPSQ,CCCCCBCCCCCCCCCCCCCC,CCCCCECCCCCCCCCCCCCC,20,False,XRAY,1.90,0.23,0.27
1,1LM8,H,DLDLEMLAPYIPMDDDFQLR,CCCCCCCCCBCCSCCCEECC,CCCCCCCCCECCCCCCEECC,20,False,XRAY,1.85,0.20,0.24
2,1O06,A,EEDPDLKAAIQESLREAEEA,CCCHHHHHHHHHHHHHHHTC,CCCHHHHHHHHHHHHHHHCC,20,False,XRAY,1.45,0.19,0.22
3,1QOW,D,CTFTLPGGGGVCTLTSECI*,CCTTSCTTCSSTTSSTTCCC,CCCCCCCCCCCCCCCCCCCC,20,True,XRAY,1.06,0.14,1.00
4,1RDQ,I,TTYADFIASGRTGRRNAIHD,CHHHHHHTSSCSSCCCCEEC,CHHHHHHCCCCCCCCCCEEC,20,False,XRAY,1.26,0.13,0.16


In [5]:
# Prepare data for training
X = casp11_data['seq']  # Amino acid sequence
y = casp11_data['sst3']  # Secondary structure labels (C, B, H)

# Split the dataset into training, validation, and test sets
# Do not stratify on the full sequence as each sequence is unique
train_texts, val_texts, train_labels, val_labels = train_test_split(X, y, test_size=0.2, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(val_texts, val_labels, test_size=0.5, random_state=42)

In [6]:
# Modify the model to use ESM2
# model_name = "facebook/esm2_t6_8M_UR50D"  # ESM2 model

model_name = "distilbert-base-uncased"  # DistilBERT model

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenization function
def tokenize_data(texts, tokenizer, max_length=128):
    return tokenizer(
        list(texts),
        max_length=max_length,
        truncation=True,
        padding=True,
        return_tensors="pt"  # PyTorch tensors
    )

# Tokenize the datasets
train_encodings = tokenize_data(train_texts, tokenizer)
val_encodings = tokenize_data(val_texts, tokenizer)
test_encodings = tokenize_data(test_texts, tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
from datasets import Dataset
import torch

# Ensure labels are correctly mapped to integers
label_map = {'C': 0, 'B': 1, 'H': 2, 'E': 0} # Assuming 'E' maps to Coil (0)

# Convert string labels (sequences) to a list of numeric labels for each residue and pad/truncate them
def prepare_labels(label_sequences, label_map, max_length, pad_value=-100):
    numeric_labels = []
    for seq in label_sequences:
        numeric_seq = [label_map.get(char, pad_value) for char in seq] # Use .get with pad_value for unknown chars
        # Truncate numeric sequence if longer than max_length
        truncated_seq = numeric_seq[:max_length]
        # Pad the numeric sequence
        padded_seq = truncated_seq + [pad_value] * (max_length - len(truncated_seq))
        numeric_labels.append(padded_seq)
    return numeric_labels

# Get max_length from tokenization for each split
train_max_length = train_encodings["input_ids"].shape[1]
val_max_length = val_encodings["input_ids"].shape[1]
test_max_length = test_encodings["input_ids"].shape[1]


train_labels_numeric_padded = prepare_labels(train_labels, label_map, train_max_length)
val_labels_numeric_padded = prepare_labels(val_labels, label_map, val_max_length)
test_labels_numeric_padded = prepare_labels(test_labels, label_map, test_max_length)


# Convert the data into Hugging Face Dataset format with torch tensors
train_data = Dataset.from_dict({
    "input_ids": train_encodings["input_ids"].clone().detach(),
    "attention_mask": train_encodings["attention_mask"].clone().detach(),
    "labels": torch.tensor(train_labels_numeric_padded, dtype=torch.long) # Use the padded numeric labels
})

val_data = Dataset.from_dict({
    "input_ids": val_encodings["input_ids"].clone().detach(),
    "attention_mask": val_encodings["attention_mask"].clone().detach(),
    "labels": torch.tensor(val_labels_numeric_padded, dtype=torch.long) # Use the padded numeric labels
})

test_data = Dataset.from_dict({
    "input_ids": test_encodings["input_ids"].clone().detach(),
    "attention_mask": test_encodings["attention_mask"].clone().detach(),
    "labels": torch.tensor(test_labels_numeric_padded, dtype=torch.long) # Use the padded numeric labels
})

In [9]:
from transformers import AutoModelForTokenClassification, TrainingArguments # Added import here

# Function to define the model and training arguments for Optuna optimization
def create_model(params):
    # Create an ESM2 model for Token Classification
    model = AutoModelForTokenClassification.from_pretrained(
        model_name,
        num_labels=3  # 3 classes (C, B, H). Note: 'E' was mapped to 'C' (0)
    )

    # Training arguments with hyperparameters to tune
    training_args = TrainingArguments(
        output_dir=f"./esm2_token_output", # Changed output directory name
        eval_strategy="epoch",
        learning_rate=params['learning_rate'],
        per_device_train_batch_size=params['batch_size'],
        per_device_eval_batch_size=32,
        num_train_epochs=params['epochs'],
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=10,
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1", # Still using F1 as a metric
        greater_is_better=True,
        report_to="none",
    )
    return model, training_args

In [10]:
# Load evaluation metrics
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
accuracy_metric = evaluate.load("accuracy")

# Define the compute_metrics function to calculate accuracy, precision, recall, F1, and Q3 accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Flatten the predictions and labels and remove padding (label -100)
    flat_predictions = predictions.flatten()
    flat_labels = labels.flatten()

    # Filter out padding
    mask = flat_labels != -100
    filtered_predictions = flat_predictions[mask]
    filtered_labels = flat_labels[mask]

    # Compute Accuracy, Precision, Recall, F1, and Macro F1 on filtered data
    accuracy = accuracy_metric.compute(predictions=filtered_predictions, references=filtered_labels)
    precision = precision_metric.compute(predictions=filtered_predictions, references=filtered_labels, average="weighted")
    recall = recall_metric.compute(predictions=filtered_predictions, references=filtered_labels, average="weighted")
    f1 = f1_metric.compute(predictions=filtered_predictions, references=filtered_labels, average="weighted")

    # Calculate Q3 accuracy (percentage of correctly predicted residues in the sequence)
    # Q3 accuracy is calculated on all residues excluding padding
    q3_correct = sum(filtered_predictions == filtered_labels)
    q3_accuracy = q3_correct / len(filtered_labels) if len(filtered_labels) > 0 else 0.0


    return {
        "accuracy": accuracy["accuracy"],
        "precision": precision["precision"],
        "recall": recall["recall"],
        "f1": f1["f1"],
        "q3_accuracy": q3_accuracy
    }

In [11]:
# Function for hyperparameter optimization
def objective(trial):
    # Define hyperparameters to tune
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 6e-5),
        'batch_size': trial.suggest_int('batch_size', 8, 32),
        'epochs': trial.suggest_int('epochs', 3, 15)
    }

    # Create model and training args with hyperparameters
    model, training_args = create_model(params)

    # Trainer setup
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data,
        eval_dataset=val_data,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Early stopping
    )

    # Train the model
    trainer.train()

    # Evaluate the model using the Trainer's evaluation method which uses compute_metrics
    eval_results = trainer.evaluate(val_data)

    # Get the F1 score from the evaluation results
    f1 = eval_results["eval_f1"] # Access the f1 metric calculated by compute_metrics

    return f1

In [12]:
# Set up Optuna study and perform hyperparameter optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=3)

# Output the best hyperparameters
print("Best hyperparameters: ", study.best_trial.params)

[I 2025-06-27 19:59:00,036] A new study created in memory with name: no-name-601252a4-0f3f-4bfa-a811-0353727b6c07
/tmp/ipython-input-11-219990905.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 6e-5),


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-11-219990905.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Q3 Accuracy
1,0.590300,0.603402,0.678701,0.685540,0.678701,0.586793,0.678701
2,0.621300,0.606117,0.681298,0.688997,0.681298,0.593260,0.681298
3,0.593900,0.605339,0.679610,0.695839,0.679610,0.585090,0.679610
4,0.604500,0.600987,0.683473,0.694952,0.683473,0.596892,0.683473
5,0.583400,0.606213,0.683246,0.690136,0.683246,0.598685,0.683246
6,0.603700,0.604968,0.682905,0.685533,0.682905,0.600306,0.682905
7,0.609600,0.607516,0.682483,0.682841,0.682483,0.600668,0.682483
8,0.579400,0.612805,0.682093,0.686580,0.682093,0.597079,0.682093
9,0.568500,0.617661,0.682012,0.683761,0.682012,0.598518,0.682012
10,0.548600,0.617349,0.681753,0.683511,0.681753,0.597823,0.681753


[I 2025-06-27 20:12:47,374] Trial 0 finished with value: 0.6006677219661027 and parameters: {'learning_rate': 1.5035829981963632e-05, 'batch_size': 8, 'epochs': 10}. Best is trial 0 with value: 0.6006677219661027.
/tmp/ipython-input-11-219990905.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 6e-5),
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-11-219990905.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` ins

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Q3 Accuracy
1,0.607900,0.603469,0.679123,0.681399,0.679123,0.590382,0.679123
2,0.627900,0.605433,0.680227,0.690325,0.680227,0.589295,0.680227
3,0.598900,0.607875,0.678409,0.701416,0.678409,0.579643,0.678409
4,0.607500,0.601654,0.682564,0.694885,0.682564,0.594232,0.682564
5,0.601500,0.606748,0.682645,0.696589,0.682645,0.593699,0.682645
6,0.563900,0.611799,0.682451,0.687593,0.682451,0.597629,0.682451
7,0.585900,0.611776,0.682077,0.683454,0.682077,0.598927,0.682077
8,0.571000,0.617117,0.682337,0.686231,0.682337,0.598059,0.682337


[I 2025-06-27 20:22:37,922] Trial 1 finished with value: 0.5989270354564544 and parameters: {'learning_rate': 2.6914287853801783e-05, 'batch_size': 12, 'epochs': 8}. Best is trial 0 with value: 0.6006677219661027.
/tmp/ipython-input-11-219990905.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 6e-5),
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-11-219990905.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` ins

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Q3 Accuracy
1,0.596700,0.606520,0.678116,0.661549,0.678116,0.603695,0.678116
2,0.607100,0.601341,0.681201,0.675101,0.681201,0.602005,0.681201
3,0.610800,0.600020,0.682304,0.685671,0.682304,0.598289,0.682304


[I 2025-06-27 20:25:46,248] Trial 2 finished with value: 0.6036952016334665 and parameters: {'learning_rate': 5.294640115502486e-05, 'batch_size': 19, 'epochs': 3}. Best is trial 2 with value: 0.6036952016334665.


Best hyperparameters:  {'learning_rate': 5.294640115502486e-05, 'batch_size': 19, 'epochs': 3}


In [13]:
# Now train the model with the best hyperparameters from Optuna
best_params = study.best_trial.params

# Create the model and training arguments directly using the best hyperparameters
model, training_args = create_model(best_params)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# Train the final model with the best hyperparameters
trainer.train()

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-13-1493347141.py:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Q3 Accuracy
1,0.595900,0.604165,0.678701,0.665690,0.678701,0.601218,0.678701
2,0.608300,0.601876,0.680697,0.674036,0.680697,0.601053,0.680697
3,0.609900,0.599885,0.681947,0.681767,0.681947,0.599604,0.681947


TrainOutput(global_step=1149, training_loss=0.609170671750194, metrics={'train_runtime': 179.7269, 'train_samples_per_second': 121.217, 'train_steps_per_second': 6.393, 'total_flos': 439199867976300.0, 'train_loss': 0.609170671750194, 'epoch': 3.0})

In [15]:
from sklearn.metrics import classification_report
import numpy as np

# Evaluate the model on the test data using the trainer's evaluate method
eval_results = trainer.evaluate(test_data)

# The eval_results dictionary contains the metrics calculated by compute_metrics, including Q3 accuracy and f1
q3_accuracy = eval_results["eval_q3_accuracy"]
f1_score = eval_results["eval_f1"]

# To get the classification report, we need the flattened, non-padded true and predicted labels from the test set
# We can get the predictions and labels using trainer.predict and then filter
predictions = trainer.predict(test_data)
predicted_labels = np.argmax(predictions.predictions, axis=-1)
true_labels = predictions.label_ids

# Flatten and filter out padding (-100)
flat_predicted_labels = predicted_labels.flatten()
flat_true_labels = true_labels.flatten()

mask = flat_true_labels != -100
filtered_predicted_labels = flat_predicted_labels[mask]
filtered_true_labels = flat_true_labels[mask]


# Print Q3 accuracy and F1 score
print("Q3 Accuracy: {:.4f}".format(q3_accuracy))
print("Weighted F1 Score: {:.4f}".format(f1_score))


# Print classification report
# Define target names based on your label_map
target_names = ["Coil", "Sheet", "Helix"] # Assuming 0: Coil, 1: Sheet, 2: Helix
# Explicitly specify the labels to report on
report_labels = [0, 1, 2] # Corresponding to Coil, Sheet, Helix

print("\nClassification Report:")
print(classification_report(filtered_true_labels, filtered_predicted_labels, labels=report_labels, target_names=target_names))

Q3 Accuracy: 0.6919
Weighted F1 Score: 0.6023

Classification Report:
              precision    recall  f1-score   support

        Coil       0.70      0.97      0.81     41669
       Sheet       0.00      0.00      0.00         0
       Helix       0.56      0.08      0.14     19071

    accuracy                           0.69     60740
   macro avg       0.42      0.35      0.32     60740
weighted avg       0.66      0.69      0.60     60740



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me